In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

imports the fastbook library and sets up the notebook for use with the fastbook library.

In [ ]:
#hide
from fastbook import *

imports all the necessary functions from fastbook.

In [ ]:
path = untar_data(URLs.PETS)/'images'
def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=21,
    label_func=is_cat, item_tfms=Resize(224))
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

Sets the path to the PETS dataset. Defines a function is_cat to check if an image is a cat based on the naming convention of the images. Creates an ImageDataLoaders object for loading the image data. Creates a vision_learner object using the ResNet34 architecture. Fine-tunes the model for one epoch.

epoch	train_loss	valid_loss	error_rate	time
0	0.145994	0.019272	0.006089	00:14
epoch	train_loss	valid_loss	error_rate	time
0	0.053405	0.052540	0.010825	00:19

In [ ]:
img = PILImage.create(image_cat())
x, = first(dls.test_dl([img]))

loads an image of a cat and gets the first item from the test dataloader.

In [ ]:
class Hook():
    def hook_func(self, m, i, o): self.stored = o.detach().clone()

defines a Hook class to capture the output of a layer during forward pass.

In [ ]:
hook_output = Hook()
hook = learn.model[0].register_forward_hook(hook_output.hook_func)

registers the hook to the last convolutional layer of the model.

In [ ]:
with torch.no_grad(): output = learn.model.eval()(x)

gets the output of the model for the input image.

In [ ]:
act = hook_output.stored[0]

gets the activation map from the stored output of the hook.

In [ ]:
F.softmax(output, dim=-1)

prints the softmax output of the model.

tensor([[0.0010, 0.9990]], device='cuda:0')

In [ ]:
dls.vocab

prints the vocabulary (categories) of the data loaders.

(#2) [False,True]

In [ ]:
x.shape

(#2) [False,True]


prints the shape of the input image.

In [ ]:
cam_map = torch.einsum('ck,kij->cij', learn.model[1][-1].weight, act)
cam_map.shape

torch.Size([2, 7, 7])

calculates the class activation map (CAM).

In [ ]:
x_dec = TensorImage(dls.train.decode((x,))[0][0])
_,ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map[1].detach().cpu(), alpha=0.6, extent=(0,224,224,0),
              interpolation='bilinear', cmap='magma');

displays the original image and overlays the CAM.

In [ ]:
hook.remove()

removes the hook.

In [ ]:
class Hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)   
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

In [ ]:
with Hook(learn.model[0]) as hook:
    with torch.no_grad(): output = learn.model.eval()(x.cuda())
    act = hook.stored

In [ ]:
class HookBwd():
    def __init__(self, m):
        self.hook = m.register_backward_hook(self.hook_func)   
    def hook_func(self, m, gi, go): self.stored = go[0].detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

In [ ]:
cls = 1
with HookBwd(learn.model[0]) as hookg:
    with Hook(learn.model[0]) as hook:
        output = learn.model.eval()(x.cuda())
        act = hook.stored
    output[0,cls].backward()
    grad = hookg.stored

In [ ]:
w = grad[0].mean(dim=[1,2], keepdim=True)
cam_map = (w * act[0]).sum(0)

In [ ]:
_,ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
              interpolation='bilinear', cmap='magma');

In [ ]:
with HookBwd(learn.model[0][-2]) as hookg:
    with Hook(learn.model[0][-2]) as hook:
        output = learn.model.eval()(x.cuda())
        act = hook.stored
    output[0,cls].backward()
    grad = hookg.stored

In [ ]:
w = grad[0].mean(dim=[1,2], keepdim=True)
cam_map = (w * act[0]).sum(0)

In [ ]:
_,ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
              interpolation='bilinear', cmap='magma');

Now this repeat the process of registering a hook, getting the output and gradient, and displaying the CAM, but this time for the penultimate layer of the model.